In [43]:
def convert_to_datstring(dataframe_,columnName_='Date',format_= "%d/%m/%Y",ref_day_ ='01'):
    ''' during emdat_csv_formated_edited.csv file reading the 'Date'
    column has some random zero days and months values'''
    for i,dat in enumerate(dataframe_.loc[:,columnName_]):
        try:
            dat = dat.strip()  #strip removes spaces from both left and right of an argument
            dataframe_.loc[i,'NewDate']=datetime.strptime(dat,format_)
        except:
            dat = dat.strip().split(r'/')
            dat[0]=ref_day_ 
            dat[1]=ref_day_
            dat="/".join(dat)
            dataframe_.loc[i,'NewDate']=datetime.strptime(dat,format_)
    return dataframe_

In [60]:
s = '1/2/2009 '
a=s.strip().split('/')
k="-".join(a)
k[4:9]

'2009'

In [45]:
s ='1/2/2009 '
l = s.strip().replace('/','-')
l

'1-2-2009'

In [46]:
import pandas as pd
from datetime import date, datetime, timedelta
import numpy as np
emdat_ep=pd.read_csv('emdat_csv_formated_edited.csv')

In [47]:
emdat_ep_flood = emdat_ep[emdat_ep.Type.isin(['Drought'])]
emdat_ep_cholera = emdat_ep[emdat_ep.Name.isin(['nf'])]

disease_country = emdat_ep_cholera[(emdat_ep_cholera.Country == 'Australia')]
disease_flood = emdat_ep_flood[(emdat_ep_flood.Country == 'Afghanistan')]

In [48]:
convert_to_datstring(dataframe_= disease_flood)

,Unnamed: 0,Date,Country,Type,Sub Type,Name,Year,Tot. Affected,NewDate
0,0,00/00/2008,Afghanistan,Drought,Drought,NaN,2008.0,280000.0,2008-01-01
1,1,00/00/2006,Afghanistan,Drought,Drought,NaN,2006.0,1900000.0,2006-01-01
2,2,00/08/2002,Afghanistan,Drought,Drought,NaN,2000.0,2580000.0,2002-01-01
3,3,00/07/1973,Afghanistan,Drought,Drought,NaN,1971.0,NaN,1973-01-01
4,4,00/05/1969,Afghanistan,Drought,Drought,NaN,1969.0,48000.0,1969-01-01


In [42]:
import pandas as pd
from datetime import date, datetime, timedelta
import numpy as np
from __future__ import division
emdat_ep=pd.read_csv('emdat_csv_formated_edited.csv')
#con = (emdat_ep['Country'].tolist())

emdat_ep_flood = emdat_ep[emdat_ep.Type.isin(['Flood'])]
emdat_ep_cholera = emdat_ep[emdat_ep.Name.isin(['nf'])]

disease_country = emdat_ep_cholera[(emdat_ep_cholera.Country == 'Afghanistan')]
disease_flood = emdat_ep_flood[(emdat_ep_flood.Country == 'Afghanistan')]
for j in emdat_ep_cholera.Country:
#    disease_country=j.reset_index(drop=True)
    disease_country_dat = convert_to_datstring(dataframe_= disease_country)
    for k in emdat_ep_flood.Country:
#        disease_flood=k.reset_index(drop=True)
        disease_flood_dat = convert_to_datstring(dataframe_= disease_flood)
            
            ################################################
            #print disease_country_dat
        df = pd.concat([disease_flood_dat, disease_country_dat], axis=1)
        df = df.drop(['Date','Type','Sub Type','Year','Name','Unnamed: 0','Unnamed: 0'], axis=1)
        df.columns = ['country_flood','flood_date','country_cholera','cholera_date']
        week = timedelta(days = 15)
        timespan=np.arange(0,28)
        res_vec=np.zeros((timespan.shape[0],3))
        for n in timespan:
            borders= pd.concat([df.flood_date + n*week, df.flood_date + (n+1)*week], axis=1)
            borders.columns=['left_border','right_border']
            counts = 0
            total_occ_cholera=0
            for i, dis in enumerate(df.cholera_date):
                #cholera_date is sorted such that null values are at the end
                if pd.isnull(dis):
                    break
                else:
                        total_occ_cholera +=1
                for flood in df.flood_date:
                    right_b, left_b = borders.right_border[i],borders.left_border[i]
                    if flood < right_b and flood >= left_b:
                        #   print "yes", dis, left_b, right_b
                        counts += 1
                        break
                res_vec[n,0] = counts
                res_vec[n,1] = total_occ_cholera
                res_vec[n,2] = counts/total_occ_cholera
    
    #  print res_vec
      #  print res_vec.shape

df_res = pd.DataFrame(res_vec)
df_res = df_res.T
df_res.to_csv('f.csv')

SyntaxError: from __future__ imports must occur at the beginning of the file (<ipython-input-42-ad56342d8d57>, line 7)